In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-04-29 04:20:19--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-04-29 04:20:19 (44.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/parse-faq.ipynb

--2025-04-29 04:23:15--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/parse-faq.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5860 (5.7K) [text/plain]
Saving to: ‘parse-faq.ipynb’

parse-faq.ipynb     100%[===================>]   5.72K  --.-KB/s    in 0s      

2025-04-29 04:23:15 (44.5 MB/s) - ‘parse-faq.ipynb’ saved [5860/5860]



In [4]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

--2025-04-29 04:24:44--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.02s   

2025-04-29 04:24:44 (33.3 MB/s) - ‘documents.json’ saved [658332/658332]



In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
docs_raw[1]['documents'][0].keys()

dict_keys(['text', 'section', 'question'])

In [5]:
#docs_raw[0]['documents'][0]

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
#documents[0]

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
index.fit(documents)

In [11]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [12]:
res = search(q)

In [1]:
#res

In [13]:
from openai import OpenAI

In [14]:
client = OpenAI()

In [15]:
response = client.chat.completions.create(
    model='gpt-4.1',
    messages=[{"role": "user", "content": q}]
)
response.choices[0].message.content

"It depends on the course and the institution offering it. Some courses may allow late enrollment, especially if only a short amount of time has passed since the start date. Others may have strict deadlines and do not permit new students to join after the course has begun.\n\nHere are some steps you can take:\n\n1. **Check the enrollment policy:** Visit the course website or contact the admissions office to see if late enrollment is possible.\n2. **Contact the instructor or coordinator:** They may grant exceptions or provide advice on catching up if late enrollment is allowed.\n3. **Consider the workload:** If you're allowed to join late, make sure you can catch up on missed materials and assignments.\n\nLet me know which course and institution you’re referring to for more specific guidance!"

In [29]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [16]:
#prompt_template

In [23]:
context = ""
for doc in res:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [24]:
#print(context)

In [25]:
prompt = prompt_template.format(question=q, context=context).strip()

In [26]:
#print(prompt)

In [27]:
response = client.chat.completions.create(
        model='gpt-4.1',
        messages=[{"role": "user", "content": prompt}]
    )

In [28]:
response.choices[0].message.content

"Yes, you can still enroll even after the course has started. You are eligible to submit the homeworks even if you register late or don't register at all. However, please note that there will be deadlines for turning in the final projects, so make sure not to leave everything for the last minute."

## modularize the code  above## 

In [42]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [43]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [44]:
query = 'the course has already started, can I still enroll?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [45]:
rag(query)

'Yes, you can still enroll in the course even if it has already started. You are also eligible to submit the homework assignments, but keep in mind the deadlines for turning in the final projects to avoid last-minute submissions.'

In [47]:
rag('how do I run kafka?')

'To run Kafka in a Java project, you can execute the following command in the project directory:\n\n```sh\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'